In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from datetime import timedelta
from dateutil.parser import parse
import missingno as msno

%matplotlib inline
%config InlineBackend.figure_format='retina'

plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'NanumGothic'

In [13]:
pip install implicit umap-learn -q

Note: you may need to restart the kernel to use updated packages.


In [15]:
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
from umap import UMAP
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split

In [29]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve
import random
import implicit
from sklearn.preprocessing import MinMaxScaler

In [2]:
# parquet_path = "/content/drive/MyDrive/Colab Notebooks/Datasets/cp2_ecommerce/df_mv_filled.parquet"
parquet_path = "D:/cp2_dataset/df_cat_add.parquet"
df = pd.read_parquet(parquet_path, engine='pyarrow')

In [3]:
df.head(3)

,event_time,month,day,day_name,hour,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,main_cat,sub_cat_1,sub_cat_2
0,2019-10-01 04:00:00+04:00,10,1,Tuesday,4,view,44600062,2103807459595387724,no_cat,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c,no_cat,None,None
1,2019-10-01 04:00:00+04:00,10,1,Tuesday,4,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc,appliances,environment,water_heater
2,2019-10-01 04:00:01+04:00,10,1,Tuesday,4,view,17200506,2053013559792632471,furniture.living_room.sofa,no_brand,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8,furniture,living_room,sofa


In [3]:
cart_cond = (df["event_type"] == "cart")
df_cart = df[cart_cond]

In [4]:
df.shape, df_cart.shape

((42380091, 16), (926366, 16))

In [49]:
df_cart.head(3)

,event_time,month,day,day_name,hour,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,main_cat,sub_cat_1,sub_cat_2
331,2019-10-01 04:05:14+04:00,10,1,Tuesday,4,cart,5100816,2053013553375346967,electronics.clocks,xiaomi,29.51,550121407,6f623695-9581-4633-813f-825b8760c7ae,electronics,clocks,None
583,2019-10-01 04:09:33+04:00,10,1,Tuesday,4,cart,1002524,2053013555631882655,electronics.smartphone,apple,515.67,524325294,0b74a829-f9d7-4654-b5b0-35bc9822c238,electronics,smartphone,None
680,2019-10-01 04:11:00+04:00,10,1,Tuesday,4,cart,4804056,2053013554658804075,electronics.audio.headphone,apple,161.98,533624186,e5ac3caa-e6d5-4d6b-ae06-2c18cd9ca683,electronics,audio,headphone


In [48]:
sub_cat_1 = (df["sub_cat_1"] == "smartphone") \
            | (df["sub_cat_1"] == "video") \
            | (df["sub_cat_1"] == "notebook") \
            | (df["sub_cat_1"] == "kitchen")

day_cond = (df["day_name"] == "Friday") \
           | (df["day_name"] == "Saturday") \
           | (df["day_name"] == "Sunday") 

In [50]:
sub_cat_1 = (df["sub_cat_1"] == "smartphone") \
            | (df["sub_cat_1"] == "video") \
            | (df["sub_cat_1"] == "notebook") \
            | (df["sub_cat_1"] == "kitchen")
df_cart_cat = df_cart.loc[sub_cat_1]
df_day = df_cart_cat.loc[day_cond]
df_day.shape

(296936, 16)

In [51]:
df_prac = df_day[["user_id", "product_id"]]
df_prac

,user_id,product_id
3515590,512433763,1005136
3515915,556510717,1003310
3516066,514302701,1005141
3516110,550696493,1004945
3516183,515905214,1005118
...,...,...
42447907,542774966,1004767
42448124,566280291,1005134
42448135,566280291,1005134
42448156,512717356,1003306


In [52]:
df_prac.loc[:, "quantity"] = 1

c:\Users\aryij\anaconda3\envs\cp2\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [53]:
df_prac

,user_id,product_id,quantity
3515590,512433763,1005136,1
3515915,556510717,1003310,1
3516066,514302701,1005141,1
3516110,550696493,1004945,1
3516183,515905214,1005118,1
...,...,...,...
42447907,542774966,1004767,1
42448124,566280291,1005134,1
42448135,566280291,1005134,1
42448156,512717356,1003306,1


In [54]:
df_grouped = df_prac.groupby(["user_id", "product_id"])["quantity"].sum().reset_index()
df_grouped.head(3)

,user_id,product_id,quantity
0,284344819,1005122,1
1,318611205,1005003,1
2,336595257,1004767,2


In [55]:
customers = list(np.sort(df_grouped['user_id'].unique()))
products = list (df_grouped['product_id'].unique())
quantity = list(df_grouped['quantity'])

rows = df_grouped['user_id'].astype('category').cat.codes
cols = df_grouped['product_id'].astype('category').cat.codes

print(len(customers)) # 126195
print(len(products))  # 3229

# csr: Compressed Sparse matrix by Row
purchase_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape = (len(customers),len(products)))
purchase_sparse #126195 x 3229 행렬

126195
3229


<126195x3229 sparse matrix of type '<class 'numpy.intc'>'
	with 168327 stored elements in Compressed Sparse Row format>

In [56]:
# Sparsity: 얼마나 비어있나?
matrix_size = purchase_sparse.shape[0]* purchase_sparse.shape[1]
num_purchases = len(purchase_sparse.nonzero()[0])
sparsity = 100 * (1 - (num_purchases / matrix_size))
sparsity

# 상호작용 행렬의 99.9%가 비어있다
# sparsity가 99.5%를 넘기 때문에, KNN을 사용하면 안되고 MF를 사용해야 한다

99.95869110381864

In [57]:
# Train, Validation 세트 생성

def make_train (matrix, percentage = .2):
    '''
    -----------------------------------------------------
    설명
    유저-아이템 행렬 (matrix)에서 
    1. 0 이상의 값을 가지면 1의 값을 갖도록 binary하게 테스트 데이터를 만들고
    2. 훈련 데이터는 원본 행렬에서 percentage 비율만큼 0으로 바뀜
    
    -----------------------------------------------------
    반환
    training_set: 훈련 데이터에서 percentage 비율만큼 0으로 바뀐 행렬
    test_set:     원본 유저-아이템 행렬의 복사본
    user_inds:    훈련 데이터에서 0으로 바뀐 유저의 index
    '''
    test_set = matrix.copy()
    test_set[test_set !=0] = 1 # binary하게 만들기
    
    training_set = matrix.copy()
    nonzero_inds = training_set.nonzero()
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1]))
    
    random.seed(0)
    num_samples = int(np.ceil(percentage * len(nonzero_pairs)))
    samples = random.sample (nonzero_pairs, num_samples)
    
    user_inds = [index[0] for index in samples]
    item_inds = [index[1] for index in samples]
    
    training_set[user_inds, item_inds] = 0
    training_set.eliminate_zeros()
    
    return training_set, test_set, list(set(user_inds))

In [58]:
product_train, product_test, product_users_altered = make_train(purchase_sparse, 0.2)

In [75]:
product_train

<126195x3229 sparse matrix of type '<class 'numpy.intc'>'
	with 134661 stored elements in Compressed Sparse Row format>

In [76]:
product_test

<126195x3229 sparse matrix of type '<class 'numpy.intc'>'
	with 168327 stored elements in Compressed Sparse Row format>

In [81]:
from implicit.als import AlternatingLeastSquares

model_user_item = AlternatingLeastSquares(factors=128,
                                          random_state=42,
                                          iterations=10,
                                          calculate_training_loss=True,
                                         )
model_user_item.fit(product_train)

c:\Users\aryij\anaconda3\envs\cp2\lib\site-packages\implicit\utils.py:26: UserWarning: OpenBLAS detected. Its highly recommend to set the environment variable 'export OPENBLAS_NUM_THREADS=1' to disable its internal multithreading
  warnings.warn(
100%|██████████| 10/10 [00:33<00:00,  3.33s/it, loss=0.000147]


In [82]:
# ALS Model

In [84]:
user_id = 515905214

In [96]:
user_idx = df_prac.index[df_prac["user_id"] == user_id]

In [100]:
user_idx

Int64Index([3516183, 3517069, 4212996, 4268014, 34692442, 34693866], dtype='int64')

In [99]:
recommendation = model_user_item.recommend(user_idx, product_train[user_idx], N=20)

products.loc[[k[0] for k in recommendation], :]

IndexError: index (34693866) out of range

In [19]:
class RandomModel():
    def __init__(self):
        self.products = None
    
    def train(self, products):
        self.products = np.asarray(products)
        return self

    def recommend(self, size=(1,10)):
        return np.random.choice(self.products, size=size, replace=False)

In [20]:
random_model = RandomModel().train(df.index.tolist())

In [21]:
random_model

In [24]:
random_rec = random_model.recommend()

In [25]:
df.loc[random_rec[0].tolist(), :]

,event_time,month,day,day_name,hour,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,main_cat,sub_cat_1,sub_cat_2
3155145,2019-10-03 18:49:44+04:00,10,3,Thursday,18,view,23300447,2053013561956893455,missing,missing,33.15,548792362,302ddeac-081f-4397-bf86-8440e8327d1b,missing,None,None
32645141,2019-10-24 15:41:55+04:00,10,24,Thursday,15,view,13901042,2053013557343158789,construction.components.faucet,no_brand,26.92,533443539,36dccc67-f8fa-46f4-8fc7-65ab33b72e25,construction,components,faucet
16671172,2019-10-13 15:50:36+04:00,10,13,Sunday,15,view,18700066,2053013555380224399,no_cat,sv,205.90,514487409,acea6c1b-1361-4939-b068-b92e726408a4,no_cat,None,None
18512307,2019-10-14 19:48:27+04:00,10,14,Monday,19,view,43900057,2127425440925614628,no_cat,bosch,123.04,516076072,ea16b310-ebcf-4972-8b52-3c70630d98b9,no_cat,None,None
7161383,2019-10-06 18:29:22+04:00,10,6,Sunday,18,view,3600666,2053013563810775923,appliances.kitchen.dishwasher,samsung,319.14,557383167,b9e1f8de-73db-4346-ba1e-897e022b32fc,appliances,kitchen,dishwasher
11369511,2019-10-09 21:40:15+04:00,10,9,Wednesday,21,view,12711054,2053013553559896355,auto.accessories.tire,tunga,39.64,549890710,530a0120-712c-40d3-bb36-6cdf08fcfddc,auto,accessories,tire
29908405,2019-10-22 15:45:14+04:00,10,22,Tuesday,15,view,3701370,2053013565983425517,appliances.environment.vacuum,lg,174.45,545834965,534ad2c4-3da3-431e-941e-7c5a431f95fd,appliances,environment,vacuum
20675268,2019-10-16 09:30:30+04:00,10,16,Wednesday,9,view,1004209,2053013555631882655,electronics.smartphone,samsung,88.80,535894680,899b4e95-4101-474f-9e55-fb25326f8d67,electronics,smartphone,None
7963965,2019-10-07 11:50:38+04:00,10,7,Monday,11,view,1002547,2053013555631882655,electronics.smartphone,apple,459.43,516206447,a794c940-414a-4299-987f-e2ace6fb48e7,electronics,smartphone,None
19552344,2019-10-15 14:25:04+04:00,10,15,Tuesday,14,view,1005104,2053013555631882655,electronics.smartphone,apple,1007.97,556282848,141c09d5-36ca-44ab-876e-99ffc5b805b8,electronics,smartphone,None
